<a href="https://colab.research.google.com/github/rpjena/random_matrix/blob/main/stock_factor_beta_signal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Factor Beta, Momentum & Mean Reversion: L/S Strategies

Three cross-sectional signals are built from a `(T x N)` stock return panel and a `(T,)` factor series:

| Strategy | Signal at time `t` | Direction |
|---|---|---|
| **Factor Beta** | Rolling OLS beta to factor `F` | Long high-beta |
| **Momentum** | Cumulative return months `[t-11, t-1]` — skip most recent | Long past winners |
| **Mean Reversion** | Negative of 1-month return `−r_i(t)` | Long recent losers |

**No double counting**: for the combined strategy each signal is cross-sectionally z-scored, averaged into a single composite score, and ranked *once* — every stock lands in exactly one of {long, neutral, short}.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')

## 1. Synthetic Data Generation

We simulate a factor model:
```
r_i(t) = beta_i(t) * F(t) + epsilon_i(t)
```
where betas are slowly time-varying (random walk) so the signal has realistic dynamics.

In [ ]:
np.random.seed(42)

T = 300   # number of time periods (monthly: ~25 years)
N = 100   # number of stocks

dates = pd.date_range('2000-01-31', periods=T, freq='ME')

# --- Factor returns: mean-zero, std ~3% per month ---
F = pd.Series(np.random.normal(0.0, 0.03, T), index=dates, name='factor')

# --- True betas: slow random walk per stock, centred around 1 ---
true_beta = np.zeros((T, N))
true_beta[0] = np.random.uniform(0.2, 1.8, N)
for t in range(1, T):
    true_beta[t] = true_beta[t-1] + np.random.normal(0, 0.02, N)

# --- Stock returns = beta * F + idiosyncratic noise ---
idio_vol = np.random.uniform(0.02, 0.06, N)          # per-stock idio vol
epsilon = np.random.normal(0, 1, (T, N)) * idio_vol  # (T x N) noise
raw_returns = true_beta * F.values[:, None] + epsilon

returns = pd.DataFrame(raw_returns, index=dates,
                       columns=[f'Stock_{i:03d}' for i in range(N)])

print(f'returns shape : {returns.shape}   (T x N)')
print(f'F shape       : {F.shape}   (T,)')
print(returns.head())

## 2. Shared Utilities

These functions are reused by all three strategies.

In [ ]:
def compute_signal(scores, k=0.2):
    """
    Build equal-weighted L/S weights from any cross-sectional score.

    Parameters
    ----------
    scores : pd.DataFrame, shape (T, N)
        Any score where higher value = expected outperformer.
    k : float
        Fraction of stocks in each leg.

    Returns
    -------
    weights : pd.DataFrame, shape (T, N)
        Long sums to +1, short sums to -1 each row.
    long_mask  : pd.DataFrame, shape (T, N), bool
    short_mask : pd.DataFrame, shape (T, N), bool
    """
    pct_rank = scores.rank(axis=1, pct=True, na_option='keep')

    long_mask  = pct_rank > (1 - k)
    short_mask = pct_rank <= k

    n_long  = long_mask.sum(axis=1)
    n_short = short_mask.sum(axis=1)

    weights = pd.DataFrame(0.0, index=scores.index, columns=scores.columns)
    for t in weights.index:
        nl, ns = n_long[t], n_short[t]
        if nl > 0:
            weights.loc[t, long_mask.loc[t]]  =  1.0 / nl
        if ns > 0:
            weights.loc[t, short_mask.loc[t]] = -1.0 / ns

    return weights, long_mask, short_mask


def compute_portfolio_returns(weights, returns):
    """
    Compute forward-looking L/S portfolio returns (signal at t -> return at t+1).

    Parameters
    ----------
    weights : pd.DataFrame, shape (T, N)
    returns : pd.DataFrame, shape (T, N)

    Returns
    -------
    strat_ret : pd.Series
    long_ret  : pd.Series
    short_ret : pd.Series
    """
    w_lagged = weights.shift(1)
    w_long   = w_lagged.clip(lower=0)
    w_short  = (-w_lagged).clip(lower=0)

    long_ret  = (w_long  * returns).sum(axis=1)
    short_ret = (w_short * returns).sum(axis=1)
    strat_ret = long_ret - short_ret

    valid = strat_ret != 0
    return strat_ret[valid], long_ret[valid], short_ret[valid]


def compute_performance(ret, freq=12, label='Strategy'):
    """
    Compute annualised performance statistics.

    Parameters
    ----------
    ret : pd.Series
    freq : int
        Periods per year (12 for monthly).
    label : str

    Returns
    -------
    stats   : dict
    cum_ret : pd.Series  Cumulative wealth index starting at 1.
    """
    cum_ret   = (1 + ret).cumprod()
    n_years   = len(ret) / freq
    ann_ret   = cum_ret.iloc[-1] ** (1 / n_years) - 1
    ann_vol   = ret.std() * np.sqrt(freq)
    sharpe    = ann_ret / ann_vol if ann_vol > 0 else np.nan
    rolling_max = cum_ret.cummax()
    max_dd    = ((cum_ret - rolling_max) / rolling_max).min()
    hit_rate  = (ret > 0).mean()

    stats = dict(ann_return=ann_ret, ann_vol=ann_vol, sharpe=sharpe,
                 max_drawdown=max_dd, hit_rate=hit_rate, n_periods=len(ret))

    print(f'\n--- {label} ---')
    print(f'  Ann. Return  : {ann_ret*100:+.2f}%')
    print(f'  Ann. Vol     : {ann_vol*100:.2f}%')
    print(f'  Sharpe Ratio : {sharpe:.3f}')
    print(f'  Max Drawdown : {max_dd*100:.2f}%')
    print(f'  Hit Rate     : {hit_rate*100:.1f}%')
    print(f'  Periods      : {len(ret)}')
    return stats, cum_ret


def compute_turnover(weights):
    """
    One-way turnover per period.

    Parameters
    ----------
    weights : pd.DataFrame, shape (T, N)

    Returns
    -------
    turnover : pd.Series  Fraction of gross portfolio that changes each period.
    """
    return weights.diff().abs().sum(axis=1) / 2


FREQ = 12   # monthly data
K    = 0.2  # top/bottom 2 deciles

## 3. Strategy A — Factor Beta Signal

At each time `t` with rolling window `W`:
```
beta_i(t) = Cov(r_i[t-W:t], F[t-W:t]) / Var(F[t-W:t])
```
Long stocks with highest estimated beta exposure, short stocks with lowest.

In [ ]:
def compute_rolling_betas(returns, F, window=60):
    """
    Estimate rolling OLS beta of each stock to factor F.

    Parameters
    ----------
    returns : pd.DataFrame, shape (T, N)
    F : pd.Series, shape (T,)
    window : int

    Returns
    -------
    beta_df : pd.DataFrame, shape (T, N)
        First (window-1) rows are NaN.
    """
    T_len, N_len = returns.shape
    beta_vals = np.full((T_len, N_len), np.nan)
    F_arr = F.values
    R_arr = returns.values

    for t in range(window - 1, T_len):
        f_win = F_arr[t - window + 1 : t + 1]
        r_win = R_arr[t - window + 1 : t + 1, :]
        f_dm  = f_win - f_win.mean()
        r_dm  = r_win - r_win.mean(axis=0)
        var_f = np.dot(f_dm, f_dm)
        if var_f < 1e-12:
            continue
        beta_vals[t, :] = f_dm @ r_dm / var_f

    return pd.DataFrame(beta_vals, index=returns.index, columns=returns.columns)


WINDOW  = 60
beta_df = compute_rolling_betas(returns, F, window=WINDOW)

wts_beta, lm_beta, sm_beta      = compute_signal(beta_df, k=K)
ret_beta, long_beta, short_beta = compute_portfolio_returns(wts_beta, returns)
stats_beta, cum_beta            = compute_performance(ret_beta, freq=FREQ,
                                                       label=f'Factor Beta (W={WINDOW}, k={K})')

### Beta diagnostics: cross-sectional spread over time

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

q10 = beta_df.quantile(0.1, axis=1)
q50 = beta_df.quantile(0.5, axis=1)
q90 = beta_df.quantile(0.9, axis=1)

ax = axes[0]
ax.fill_between(beta_df.index, q10, q90, alpha=0.3, label='10-90th pct')
ax.plot(beta_df.index, q50, color='navy', lw=1.5, label='Median beta')
ax.set_title('Cross-sectional Beta Distribution Over Time')
ax.set_ylabel('Beta')
ax.legend()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

ax = axes[1]
ax.hist(beta_df.iloc[-1].dropna(), bins=20, color='steelblue', edgecolor='white')
ax.set_title(f'Beta Distribution at t={beta_df.index[-1].date()}')
ax.set_xlabel('Beta')
ax.set_ylabel('Count')

plt.tight_layout()
plt.show()

## 4. Strategy B — Momentum & Strategy C — Mean Reversion

Both signals are purely price-based (no factor needed).

| Signal | Formula at time `t` | Interpretation |
|--------|---------------------|----------------|
| **Momentum** | $\sum_{s=t-11}^{t-1} r_i(s)$ &nbsp;(11-month cum. return, skip most recent 1 month) | Long past winners, short past losers |
| **Mean Reversion** | $-r_i(t)$ &nbsp;(negate most recent 1-month return) | Long recent losers, short recent winners |

The two signals use **non-overlapping** return windows:
- Momentum uses months `[t−11, t−1]`
- Mean reversion uses month `[t]`

so they carry independent information by construction.

In [ ]:
def compute_momentum_scores(returns, lookback=12, skip=1):
    """
    Cross-sectional momentum score (standard 12-1 month).

    At time t the score is the cumulative return over
    [t - lookback + skip, t - skip], i.e. the most recent `lookback - skip`
    periods shifted back by `skip` to avoid short-term reversal contamination.

    Parameters
    ----------
    returns : pd.DataFrame, shape (T, N)
    lookback : int  Total lookback window (default 12).
    skip : int      Periods to skip at the near end (default 1).

    Returns
    -------
    scores : pd.DataFrame, shape (T, N)
        NaN during burn-in (first lookback rows).
        Higher score = stronger past winner.
    """
    n_periods = lookback - skip          # number of months actually summed
    # rolling sum of n_periods months, then shift by skip so the
    # most recent `skip` months are excluded at formation time t
    return returns.rolling(n_periods).sum().shift(skip)


def compute_mean_reversion_scores(returns, lookback=1):
    """
    Short-term reversal (mean reversion) score.

    Score at time t = −r_i(t), i.e. the negated rolling mean return
    over the most recent `lookback` periods.  Stocks with the largest
    negative recent return receive the highest score (expected to bounce).

    Parameters
    ----------
    returns : pd.DataFrame, shape (T, N)
    lookback : int  Number of recent periods to average and negate (default 1).

    Returns
    -------
    scores : pd.DataFrame, shape (T, N)
        Higher score = stronger recent loser.
    """
    return -returns.rolling(lookback).mean()


MOM_LOOKBACK = 12
MOM_SKIP     = 1
MR_LOOKBACK  = 1

mom_scores = compute_momentum_scores(returns, lookback=MOM_LOOKBACK, skip=MOM_SKIP)
mr_scores  = compute_mean_reversion_scores(returns, lookback=MR_LOOKBACK)

print(f'Momentum  burn-in: {mom_scores.iloc[:, 0].isna().sum()} periods '
      f'(uses months t-{MOM_LOOKBACK+MOM_SKIP-1} to t-{MOM_SKIP})')
print(f'Mean Rev  burn-in: {mr_scores.iloc[:, 0].isna().sum()} period '
      f'(uses month t)')

### 4a. Individual strategy performance (standalone, no combination)

In [ ]:
# ── Momentum ──────────────────────────────────────────────────────────────────
wts_mom, lm_mom, sm_mom       = compute_signal(mom_scores, k=K)
ret_mom, long_mom, short_mom  = compute_portfolio_returns(wts_mom, returns)
stats_mom, cum_mom             = compute_performance(ret_mom, freq=FREQ,
                                                      label=f'Momentum ({MOM_LOOKBACK}-{MOM_SKIP}, k={K})')

# ── Mean Reversion ────────────────────────────────────────────────────────────
wts_mr, lm_mr, sm_mr          = compute_signal(mr_scores, k=K)
ret_mr, long_mr, short_mr     = compute_portfolio_returns(wts_mr, returns)
stats_mr, cum_mr               = compute_performance(ret_mr, freq=FREQ,
                                                      label=f'Mean Reversion ({MR_LOOKBACK}-month, k={K})')

### 4b. Signal independence: cross-sectional rank correlation

If momentum and mean reversion are correlated cross-sectionally, combining them
naively (two separate portfolios) would double-count overlapping positions.
We measure the Spearman rank correlation across stocks within each period.

In [ ]:
from scipy.stats import spearmanr

valid_idx   = mom_scores.dropna(how='all').index.intersection(
              mr_scores.dropna(how='all').index)
rank_corrs  = []
long_overlaps = []

for t in valid_idx:
    m = mom_scores.loc[t].dropna()
    r = mr_scores.loc[t].dropna()
    common = m.index.intersection(r.index)
    if len(common) < 10:
        continue
    rho, _ = spearmanr(m[common], r[common])
    rank_corrs.append(rho)

    # Fraction of long-leg stocks shared between the two strategies
    if t in lm_mom.index and t in lm_mr.index:
        nl = lm_mom.loc[t].sum()
        if nl > 0:
            overlap = (lm_mom.loc[t] & lm_mr.loc[t]).sum() / nl
            long_overlaps.append(float(overlap))

rank_corr_series = pd.Series(rank_corrs, index=valid_idx[:len(rank_corrs)])

print(f'Mean cross-sectional Spearman ρ (Mom vs MR) : {rank_corr_series.mean():.4f}')
print(f'  (0 = fully orthogonal, ±1 = perfectly correlated)')
print(f'Mean long-leg overlap fraction              : {np.mean(long_overlaps):.2%}')
print(f'  (fraction of momentum longs also in MR long leg)')

## 5. Composite Signal — No Double Counting

Running three separate portfolios in parallel allows a stock to be long in momentum
**and** long in mean reversion simultaneously, giving it twice the effective weight.
This is **double counting**.

**Solution**: merge all signals into one composite score before ranking.

```
z_mom(t)      = cross_sectional_zscore( mom_score(t) )
z_mr(t)       = cross_sectional_zscore(  mr_score(t) )
z_beta(t)     = cross_sectional_zscore(    beta(t)   )

composite(t)  = ( z_mom + z_mr + z_beta ) / 3
```

The composite is ranked **once**: every stock ends up in exactly one of
{long leg, neutral, short leg} — no double counting by construction.

In [ ]:
def combine_signals(signals_dict, k=0.2):
    """
    Merge multiple signals into one composite L/S portfolio (no double counting).

    Each signal is cross-sectionally z-scored (demean, divide by cross-sectional
    std) then averaged equally.  The composite is ranked once to produce a
    single set of portfolio weights so each stock can appear in at most one leg.

    Parameters
    ----------
    signals_dict : dict {name: pd.DataFrame (T, N)}
        Raw signal scores.  Higher value = expected outperformer.
        NaN cells are preserved and excluded from ranking.
    k : float
        Fraction of stocks in each leg (e.g. 0.2 = top/bottom 2 deciles).

    Returns
    -------
    weights    : pd.DataFrame (T, N)  combined portfolio weights
    composite  : pd.DataFrame (T, N)  composite z-score
    long_mask  : pd.DataFrame (T, N)  bool
    short_mask : pd.DataFrame (T, N)  bool
    """
    z_list = []
    for sig in signals_dict.values():
        mu = sig.mean(axis=1)                       # cross-sectional mean (T,)
        sd = sig.std(axis=1).replace(0, np.nan)     # cross-sectional std  (T,)
        z  = sig.sub(mu, axis=0).div(sd, axis=0)   # z-score
        z_list.append(z)

    # Equal-weight average of z-scores
    # Where any signal is NaN for a stock, composite is NaN for that stock/period
    composite = sum(z_list) / len(z_list)

    weights, long_mask, short_mask = compute_signal(composite, k=k)
    return weights, composite, long_mask, short_mask


# --- All three signals combined ---
signals_all = {
    'beta'          : beta_df,
    'momentum'      : mom_scores,
    'mean_reversion': mr_scores,
}

wts_all, composite_all, lm_all, sm_all = combine_signals(signals_all, k=K)
ret_all, long_all, short_all           = compute_portfolio_returns(wts_all, returns)
stats_all, cum_all                     = compute_performance(
    ret_all, freq=FREQ, label=f'Composite (Beta+Mom+MR, k={K})')

# --- Momentum + Mean Reversion only (no factor beta) ---
signals_mom_mr = {'momentum': mom_scores, 'mean_reversion': mr_scores}
wts_mm, composite_mm, lm_mm, sm_mm    = combine_signals(signals_mom_mr, k=K)
ret_mm, _, _                           = compute_portfolio_returns(wts_mm, returns)
stats_mm, cum_mm                       = compute_performance(
    ret_mm, freq=FREQ, label=f'Composite (Mom+MR only, k={K})')

### 5a. Verify: no double counting in the composite portfolio

In [ ]:
# A stock cannot be simultaneously long AND short — confirm zero overlap
overlap_periods = (lm_all & sm_all).any(axis=1).sum()
print(f'Periods with any long/short overlap : {overlap_periods}  (must be 0)')

# Show how the individual signal legs compare to the composite leg
def leg_overlap_fraction(mask_a, mask_b):
    """Fraction of long positions in mask_a that also appear in mask_b."""
    total = mask_a.sum().sum()
    return (mask_a & mask_b).sum().sum() / total if total > 0 else 0.0

print('\nLong-leg overlap vs composite (Beta+Mom+MR):')
print(f'  Beta leg ∩ composite long : {leg_overlap_fraction(lm_beta, lm_all):.2%}')
print(f'  Mom  leg ∩ composite long : {leg_overlap_fraction(lm_mom,  lm_all):.2%}')
print(f'  MR   leg ∩ composite long : {leg_overlap_fraction(lm_mr,   lm_all):.2%}')
print('\n(Values <100% reflect stocks that score well on one signal but not all three)')

# Turnover comparison
to_beta  = compute_turnover(wts_beta)
to_mom   = compute_turnover(wts_mom)
to_mr    = compute_turnover(wts_mr)
to_all   = compute_turnover(wts_all)

print('\nAverage one-way monthly turnover:')
for lbl, to in [('Beta', to_beta), ('Mom', to_mom), ('MR', to_mr), ('Composite', to_all)]:
    print(f'  {lbl:<12}: {to.mean()*100:.1f}%')

## 6. Full Strategy Comparison

In [ ]:
def _rebase(s, start):
    s = s[s.index >= start]
    return s / s.iloc[0]

# Common start = latest burn-in end across all strategies
start = max(cum_beta.index[0], cum_mom.index[0],
            cum_mr.index[0],   cum_all.index[0])

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle(f'Strategy Comparison  (k={K}, beta window={WINDOW})', fontsize=14)

# ── 1. Cumulative returns (rebased to common start) ─────────────────────────
ax = axes[0, 0]
for cum, lbl, col, ls in [
    (cum_beta, 'Factor Beta',   'navy',    '-'),
    (cum_mom,  'Momentum',      'green',   '-'),
    (cum_mr,   'Mean Reversion','orange',  '-'),
    (cum_mm,   'Mom+MR',        'purple',  '--'),
    (cum_all,  'Beta+Mom+MR',   'crimson', '--'),
]:
    _rebase(cum, start).plot(ax=ax, label=lbl, color=col, ls=ls, lw=2 if '--' in ls else 1.5)
ax.axhline(1, color='black', lw=0.8, ls=':')
ax.set_title('Cumulative Returns (rebased)')
ax.set_ylabel('Wealth Index')
ax.legend(fontsize=8)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# ── 2. Drawdowns ─────────────────────────────────────────────────────────────
ax = axes[0, 1]
for cum, lbl, col in [
    (cum_beta, 'Factor Beta',    'navy'),
    (cum_mom,  'Momentum',       'green'),
    (cum_mr,   'Mean Reversion', 'orange'),
    (cum_all,  'Beta+Mom+MR',    'crimson'),
]:
    c = _rebase(cum, start)
    dd = (c - c.cummax()) / c.cummax()
    dd.plot(ax=ax, label=lbl, color=col, lw=1.5)
ax.set_title('Drawdowns')
ax.set_ylabel('Drawdown')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y*100:.0f}%'))
ax.legend(fontsize=8)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# ── 3. Rolling 36-period Sharpe ───────────────────────────────────────────────
ax = axes[1, 0]
for ret, lbl, col, ls in [
    (ret_beta, 'Factor Beta',   'navy',    '-'),
    (ret_mom,  'Momentum',      'green',   '-'),
    (ret_mr,   'Mean Reversion','orange',  '-'),
    (ret_all,  'Beta+Mom+MR',   'crimson', '--'),
]:
    roll_sh = (ret.rolling(36).mean() / ret.rolling(36).std()) * np.sqrt(FREQ)
    roll_sh.plot(ax=ax, label=lbl, color=col, ls=ls, lw=2 if '--' in ls else 1.5)
ax.axhline(0, color='black', lw=0.8, ls=':')
ax.set_title('Rolling 36-period Sharpe Ratio')
ax.set_ylabel('Sharpe')
ax.legend(fontsize=8)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# ── 4. Monthly return distributions ──────────────────────────────────────────
ax = axes[1, 1]
for ret, lbl, col in [
    (ret_beta, 'Factor Beta',    'navy'),
    (ret_mom,  'Momentum',       'green'),
    (ret_mr,   'Mean Reversion', 'orange'),
    (ret_all,  'Beta+Mom+MR',    'crimson'),
]:
    ax.hist(ret, bins=40, alpha=0.4, color=col, label=lbl, density=True)
ax.set_title('Monthly Return Distributions')
ax.set_xlabel('Monthly Return')
ax.set_ylabel('Density')
ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

### Summary performance table

In [ ]:
rows = []
for label, st, to in [
    ('Factor Beta',           stats_beta, to_beta),
    ('Momentum',              stats_mom,  to_mom),
    ('Mean Reversion',        stats_mr,   to_mr),
    ('Composite Mom+MR',      stats_mm,   to_mr),    # same turnover approx
    ('Composite Beta+Mom+MR', stats_all,  to_all),
]:
    rows.append({
        'Strategy'  : label,
        'Ann Ret'   : f"{st['ann_return']*100:+.2f}%",
        'Ann Vol'   : f"{st['ann_vol']*100:.2f}%",
        'Sharpe'    : f"{st['sharpe']:.3f}",
        'Max DD'    : f"{st['max_drawdown']*100:.2f}%",
        'Hit Rate'  : f"{st['hit_rate']*100:.1f}%",
        'Avg TO/mo' : f"{to.mean()*100:.1f}%",
        'Periods'   : st['n_periods'],
    })

print(pd.DataFrame(rows).set_index('Strategy').to_string())

## 7. Sensitivity Analysis: Decile Threshold `k`

Sweep `k` over {0.1, 0.2, 0.3, 0.5} for each strategy and the composite.

In [ ]:
import seaborn as sns

k_vals = [0.1, 0.2, 0.3, 0.5]

sens_rows = []
for k in k_vals:
    for strat_label, scores in [
        ('Beta',        beta_df),
        ('Momentum',    mom_scores),
        ('Mean Rev',    mr_scores),
    ]:
        wts, _, _ = compute_signal(scores, k=k)
        s_ret, _, _ = compute_portfolio_returns(wts, returns)
        if len(s_ret) < 12:
            continue
        n_yr  = len(s_ret) / FREQ
        ann_r = (1 + s_ret).prod() ** (1 / n_yr) - 1
        ann_v = s_ret.std() * np.sqrt(FREQ)
        sh    = ann_r / ann_v if ann_v > 0 else np.nan
        sens_rows.append(dict(k=k, strategy=strat_label, sharpe=sh))

    # Composite (Beta+Mom+MR)
    wts_c, _, _, _ = combine_signals(signals_all, k=k)
    s_ret_c, _, _  = compute_portfolio_returns(wts_c, returns)
    if len(s_ret_c) >= 12:
        n_yr  = len(s_ret_c) / FREQ
        ann_r = (1 + s_ret_c).prod() ** (1 / n_yr) - 1
        ann_v = s_ret_c.std() * np.sqrt(FREQ)
        sh    = ann_r / ann_v if ann_v > 0 else np.nan
        sens_rows.append(dict(k=k, strategy='Composite', sharpe=sh))

sens_df = pd.DataFrame(sens_rows)
pivot   = sens_df.pivot(index='strategy', columns='k', values='sharpe')
# Reorder rows
pivot   = pivot.reindex(['Beta', 'Momentum', 'Mean Rev', 'Composite'])

fig, ax = plt.subplots(figsize=(8, 4))
sns.heatmap(pivot, annot=True, fmt='.3f', cmap='RdYlGn', center=0, ax=ax,
            linewidths=0.5, linecolor='grey')
ax.set_title('Sharpe Ratio by Strategy and Decile Threshold k')
ax.set_xlabel('k (fraction in each leg)')
ax.set_ylabel('Strategy')
plt.tight_layout()
plt.show()